**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

In [1]:
import numpy as np
import pandas as pd

#### Execute the below cell to load the datasets

In [2]:
#Loading data
books = pd.read_csv("books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

users = pd.read_csv('users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

ratings = pd.read_csv('ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.p

### Check no.of records and features given in each dataset

In [3]:
books.shape

(271360, 8)

In [4]:
users.shape

(278858, 3)

In [5]:
ratings.shape

(1149780, 3)

## Exploring books dataset

In [6]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop last three columns containing image URLs which will not be required for analysis

In [7]:
books.drop(columns=['imageUrlS', 'imageUrlM', 'imageUrlL'],inplace=True)

In [8]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


**yearOfPublication**

### Check unique values of yearOfPublication


In [9]:
books['yearOfPublication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [10]:
books[books['yearOfPublication']=='DK Publishing Inc']

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [11]:
books.drop(index=books[books['yearOfPublication']=='DK Publishing Inc'].index,axis=0,inplace=True)
books.drop(index=books[books['yearOfPublication']=='Gallimard'].index,axis=0,inplace=True)

### Change the datatype of yearOfPublication to 'int'

In [12]:
books['yearOfPublication']=books['yearOfPublication'].astype('int')

In [13]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication     int32
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [14]:
books['publisher'].dropna(axis=0,inplace=True)

## Exploring Users dataset

In [15]:
print(users.shape)
users.head()

(278858, 3)


,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### Get all unique values in ascending order for column `Age`

In [16]:
users.sort_values(by='Age')['Age'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 113., 114., 115., 116., 118., 119., 123., 124., 127.,
       128., 132., 133., 136., 137., 138., 140., 141., 143., 146., 147.,
       148., 151., 152., 156., 157., 159., 162., 168., 172., 175., 183.,
       186., 189., 199., 200., 201., 204., 207., 20

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [17]:
users['Age']=users['Age'].apply(lambda x : np.NAN if ((x <5)|(x>90)) else x)

### Replace null values in column `Age` with mean

In [18]:
users['Age'].fillna(value=users['Age'].mean(),inplace=True)

### Change the datatype of `Age` to `int`

In [19]:
users['Age']=users['Age'].astype('int')

In [20]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


## Exploring the Ratings Dataset

### check the shape

In [21]:
ratings.shape

(1149780, 3)

In [22]:
n_users = users.shape[0]
n_books = books.shape[0]

In [23]:
ratings.head(5)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [24]:
ratings.drop(index=ratings[~ratings["ISBN"].astype(str).isin(books["ISBN"].astype(str))].index,inplace=True)

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [25]:
ratings.drop(index=ratings[~ratings["userID"].astype(str).isin(users["userID"].astype(str))].index,inplace=True)

### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [26]:
ratings.drop(index=ratings[ratings['bookRating']==0].index,inplace=True)

### Find out which rating has been given highest number of times

In [27]:
print('highest rating is :',ratings['bookRating'].value_counts().keys()[0])

highest rating is : 8


### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [28]:
user_grp=ratings.groupby(['userID']).size().reset_index(name='counts')
good_users=user_grp[user_grp['counts']>100]

In [29]:
ratings.drop(index=ratings[~ratings["userID"].astype(str).isin(good_users["userID"].astype(str))].index,inplace=True)

In [30]:
ratings

,userID,ISBN,bookRating
1456,277427,002542730X,10
1458,277427,003008685X,8
1461,277427,0060006641,10
1465,277427,0060542128,7
1474,277427,0061009059,9
...,...,...,...
1147587,275970,185649814X,7
1147592,275970,1860462588,8
1147599,275970,1886411077,6
1147611,275970,3411086211,10


### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [31]:
R_df = ratings.pivot(index = 'userID', columns ='ISBN', values = 'bookRating').fillna(0)
R_df.tail()

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,...,B000092Q0A,B00009EF82,B00009NDAN,B0000DYXID,B0000T6KHI,B0000VZEJQ,B0000X8HIE,B00013AX9E,B0001I1KOG,B000234N3A
userID,,,,,,,,,,,,,,,,,,,,,
274061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


### Generate the predicted ratings using SVD with no.of singular values to be 50

In [33]:

from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)
#add the user means back to get the predicted full (eg: 5 Star, here the highest rating is 10) ratings.
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)  + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns, index = R_df.index)

In [34]:
preds_df.head()

ISBN,0000913154,0001046438,000104687X,0001047213,0001047973,000104799X,0001048082,0001053736,0001053744,0001055607,...,B000092Q0A,B00009EF82,B00009NDAN,B0000DYXID,B0000T6KHI,B0000VZEJQ,B0000X8HIE,B00013AX9E,B0001I1KOG,B000234N3A
userID,,,,,,,,,,,,,,,,,,,,,
2033,0.026890,-0.002154,-0.000463,-0.002154,-0.002154,0.004179,-0.001344,0.006087,0.006087,0.013031,...,-0.000608,-0.002854,0.045559,-0.015202,-0.077645,0.005082,0.030398,0.000567,-0.006105,0.069546
2110,-0.006960,-0.003468,0.002682,-0.003468,-0.003468,0.015414,0.000457,-0.002316,-0.002316,0.014683,...,0.014392,0.015081,0.019200,0.004620,-0.018477,0.015060,0.015576,0.014589,0.014542,-0.006879
2276,-0.010698,-0.016642,-0.002227,-0.016642,-0.016642,0.032789,-0.017195,0.011652,0.011652,0.034672,...,0.025758,0.027894,0.068648,0.012547,0.134119,0.031499,0.026292,0.026040,0.034756,-0.048534
4017,-0.019825,0.035705,0.023801,0.035705,0.035705,0.028771,0.029353,-0.000895,-0.000895,0.066415,...,-0.000900,0.002985,0.089550,-0.008591,0.015076,0.025493,0.000113,-0.000603,0.022845,-0.046487
4385,0.011206,-0.011829,0.012264,-0.011829,-0.011829,0.061316,-0.014140,-0.011418,-0.011418,0.051550,...,0.067029,0.070491,0.038518,0.030471,0.712491,0.059904,0.042555,0.064836,0.073250,0.070210


### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [35]:
userID = 2110

In [49]:
# This function return the books with the highest predicted rating that the specified user hasn’t already rated
# Take specific user row from matrix from predictions


def recommend_books(predictions_df, userID, books_df, original_ratings_df, num_recommendations):

    # Get and sort the user's predictions
    user_row_number = userID
    sorted_user_predictions = predictions_df.loc[user_row_number].sort_values(
        ascending=False)

    # Get the user's data and merge in the books information.
    user_data = original_ratings_df[original_ratings_df.userID == (userID)]
    # Added title and genres
    user_full = (user_data.merge(books_df, how='left', left_on='ISBN', right_on='ISBN').
                 sort_values(['bookRating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(
        userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(
        num_recommendations))

    # Recommend the highest predicted rating books that the user hasn't seen yet.
    recommendations = (books_df[~books_df['ISBN'].isin(user_full['ISBN'])].
                       merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                             left_on='ISBN',
                             right_on='ISBN').
                       rename(columns={user_row_number: 'Predictions'}).
                       sort_values('Predictions', ascending=False).
                       iloc[:num_recommendations, :-1]
                       )

    return user_full, recommendations, sorted_user_predictions, user_data, user_full

In [50]:
# Calling the function recommend_books will give all the predictions for a particualr user.
already_rated, predictions, sorted_user_predictions, user_data, user_full = recommend_books(preds_df, 2110, books, ratings, 10)

User 2110 has already rated 103 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [59]:

print ("Shape: ", predictions.shape)
user_full

Shape:  (10, 5)


,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
76,2110,067166865X,10,STAR TREK YESTERDAY'S SON (Star Trek: The Orig...,A.C. Crispin,1988,Audioworks
52,2110,0590109715,10,"The Andalite Chronicles (Elfangor's Journey, A...",Katherine Applegate,1997,Apple
64,2110,0590629786,10,"The Visitor (Animorphs, No 2)",K. A. Applegate,1996,Scholastic
63,2110,0590629778,10,"The Invasion (Animorphs, No 1)",K. A. Applegate,1996,Scholastic
61,2110,059046678X,10,The Yearbook,Peter Lerangis,1994,Scholastic
...,...,...,...,...,...,...,...
49,2110,0515134384,5,The Cat Who Went Up the Creek,Lilian Jackson Braun,2003,Jove Books
19,2110,037361490X,5,Age of War (Super Bolan #90),Don Pendleton,2003,Gold Eagle
4,2110,0151008116,5,Life of Pi,Yann Martel,2002,Harcourt
50,2110,0515136557,3,The Cat Who Brought Down the House,Lilian Jackson Braun,2004,Jove Books


### Get the predicted ratings for userID `2110` and sort them in descending order

In [39]:
# Instead of calling the function lets do it one by one...

# Get and sort the user's predictions
sorted_user_predictions = preds_df.loc[2110].sort_values(ascending=False)
print ("Shape: ", sorted_user_predictions.shape)
sorted_user_predictions.head(10)

Shape:  (66076,)


ISBN
059035342X    0.518431
0345384911    0.291064
0451151259    0.268592
0380759497    0.256567
0345370775    0.243697
044021145X    0.242707
0880389117    0.222648
1560768304    0.220658
0441845630    0.220658
0345317580    0.217696
Name: 2110, dtype: float64

### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [40]:
# Get the user's data from ratings dataframe
user_data = ratings[ratings.userID == (2110)]
print ("Shape: ", user_data.shape)
user_data.head(10)

Shape:  (103, 3)


,userID,ISBN,bookRating
14448,2110,0060987529,7
14449,2110,0064472779,8
14450,2110,0140022651,10
14452,2110,0142302163,8
14453,2110,0151008116,5
14455,2110,015216250X,8
14457,2110,0345260627,10
14458,2110,0345283554,10
14459,2110,0345283929,10
14460,2110,034528710X,10


In [41]:
user_data.head()

,userID,ISBN,bookRating
14448,2110,0060987529,7
14449,2110,0064472779,8
14450,2110,0140022651,10
14452,2110,0142302163,8
14453,2110,0151008116,5


In [42]:
user_data.shape

(103, 3)

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [43]:
user_full_info = (user_data.merge(books, how = 'left', left_on = 'ISBN', right_on = 'ISBN').
                     sort_values(['bookRating'], ascending=False))
print ("Shape: ", user_full_info.shape)
user_full_info.head()

Shape:  (103, 7)


,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
76,2110,067166865X,10,STAR TREK YESTERDAY'S SON (Star Trek: The Orig...,A.C. Crispin,1988,Audioworks
52,2110,0590109715,10,"The Andalite Chronicles (Elfangor's Journey, A...",Katherine Applegate,1997,Apple
64,2110,0590629786,10,"The Visitor (Animorphs, No 2)",K. A. Applegate,1996,Scholastic
63,2110,0590629778,10,"The Invasion (Animorphs, No 1)",K. A. Applegate,1996,Scholastic
61,2110,059046678X,10,The Yearbook,Peter Lerangis,1994,Scholastic


### Get top 10 recommendations for above given userID from the books not already rated by that user

In [44]:
# Recommend the highest predicted rating books that the user hasn't seen yet.
recommendations = (books[~books['ISBN'].isin(user_full['ISBN'])].
     merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
           left_on = 'ISBN',
           right_on = 'ISBN').
     rename(columns = {2110: 'Predictions'}).
     sort_values('Predictions', ascending = False).
                   iloc[:10, :-1]
                  )
print ("Shape: ", recommendations.shape)
recommendations

Shape:  (10, 5)


,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
6184,0345384911,Crystal Line,Anne McCaffrey,1993,Del Rey Books
2031,0451151259,Eyes of the Dragon,Stephen King,1988,Penguin Putnam~mass
976,0380759497,Xanth 15: The Color of Her Panties,Piers Anthony,1992,Eos
1192,0345370775,Jurassic Park,Michael Crichton,1999,Ballantine Books
455,044021145X,The Firm,John Grisham,1992,Bantam Dell Publishing Group
54653,0880389117,Flint the King (Dragonlance: Preludes),Mary Kirchoff,1990,Wizards of the Coast
95008,1560768304,"The Dragons of Krynn (Dragonlance Dragons, Vol...",Margaret Weis,1994,Wizards of the Coast
81942,0441845630,Unicorn Point (Apprentice Adept (Paperback)),Piers Anthony,1990,ACE Charter
2435,0345353145,Sphere,MICHAEL CRICHTON,1988,Ballantine Books
28591,1560760699,Kindred Spirits (Dragonlance: The Meetings Sex...,Mark Anthony,1991,Wizards of the Coast
